In [ ]:
import ebooklib
from ebooklib import epub
from bs4 import BeautifulSoup
import pandas as pd
from PyPDF2 import PdfReader
import os

# Exctracting data from EPUB

In [ ]:
book = epub.read_epub(r"..\data\raw\hpmor_ru.epub")
# get list of documents objects
items = list(book.get_items_of_type(ebooklib.ITEM_DOCUMENT))


In [ ]:
def chapter_to_str(chapter: str) -> str:
    """Function for html parsing epub file into text

    Args:
        chapter (str): chapter object

    Returns:
        str: text of the chapter
    """
    soup = BeautifulSoup(chapter.get_body_content(), "html.parser")
    text = [para.get_text() for para in soup.find_all("p")]
    return " ".join(text)


In [ ]:
texts = {}
for c in items:
    texts[c.get_name()] = chapter_to_str(c)

In [ ]:
print(" ".join(texts.values()))


In [ ]:
df = pd.DataFrame({'hpmor': " ".join(texts.values())}, index=['text']).T
df

# Extracting data from PDF

In [ ]:
# creating a pdf reader object
reader = PdfReader(r"..\data\raw\hp_original\1.Гарри_Поттер_и_философский_камень.pdf")

# printing number of pages in pdf file
print(len(reader.pages))


In [ ]:
# getting a specific page from the pdf file
page = reader.pages[2]

# extracting text from page
text = page.extract_text()
print(" ".join(text.split()))


In [ ]:
hp_1 = []
for page in reader.pages:
    text = page.extract_text()
    text = " ".join(text.split())
    hp_1.append(text)

In [ ]:
print("\n".join(hp_1))


In [ ]:
import os

# folder path
dir_path = r"..\\data\\raw\\hp_original\\"

# list file and directories
books = os.listdir(dir_path)


In [ ]:
hp = {title: [] for title in books}


In [ ]:
for title in books:
    print(f"Reading {title}")
    # creating a pdf reader object
    reader = PdfReader(dir_path + title)

    # printing number of pages in pdf file
    print(f"Number of pages: {len(reader.pages)}")
    temp = []
    for page in reader.pages:
        text = page.extract_text()
        text = " ".join(text.split())
        temp.append(text)
    hp[title] = "\n".join(temp)


In [ ]:
pd.DataFrame(hp, index=["text"]).T

In [ ]:
df = pd.concat([df, pd.DataFrame(hp, index=['text']).T], axis=0)

In [ ]:
df


# Analyze

In [ ]:
# crop DataFrame to avoid authors and translaters utterances
df = df.iloc[3:-1]

In [ ]:
# get length of every chapter
df["length"] = df.text.apply(len)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df.duplicated().sum()

In [ ]:
import matplotlib.pyplot as plt

plt.bar(list(range(0, df.shape[0])), df.length)
